# Summoners War - Arena Simulator / Speed Tuner

Summoners War is an awesome unit-battle, resource management, RPG mobile game that I have been playing since April 2015. 

The game is comprised of PvP as well as PvE content. For the early game, all it matters is PvE content but as you come to the end game PvP becomes more relevant. And for PvE content, speed tuning your units is important. 

I started this as I needed a speed tuner for my Arena team but could not find one for my purposes. So I decided to write my own. 

This simulator is based on the detailed theory you can find on https://www.reddit.com/r/summonerswar/comments/3vtfgk/indepth_guide_speedtuning_your_arena_team/ (Credits to reddit/iPodbiak)

I plan to make an HTML interface and put this on my website but for the timebeing I will just put this on Github.


In [77]:
import numpy as np
import pandas as pd

Here are the class descriptions for Monsters, Teams and Arena:

In [78]:
class Monster:
    
    def __init__(self, spd_base, spd_bonus,name):
        self.spd_base = spd_base
        self.spd_bonus = spd_bonus
        self.name = name 
        self.spd = spd_base + spd_bonus
        self.turn = 0
    def taketurn(self):
        self.turn += 1
        
class Team:
    def __init__ (self, spd_totem, spd_lead, mons, name):

        self.team_name = name
        spd_mdf = spd_totem + spd_lead
        self.mons = mons
        
        spds_base = [mon.spd_base for mon in mons]
        spds_bonus = [mon.spd_bonus for mon in mons]
        
        self.spds = [ spd + spd*spd_mdf + spds_bonus[i] for i,spd in enumerate(spds_base)]

class Arena:
    bar_full = 100
    
    def __init__(self, *args):
        
        self.total_ticks = args[-1]
        teams = args[0:-1]
        
        self.mons = []
        self.team_names = []
        self.spds = []
        
        for team in teams:
            self.mons.extend(team.mons)
            self.team_names.extend(team.team_name)
            self.spds.extend(team.spds)
            
    def simulate(self):
        
        bars = [0 for mon in self.mons]
        
        ticker = 1
        step_pct = 0.07
        tick_step = [step_pct*spd for spd in self.spds]
        df = pd.DataFrame(columns = [ 'Mover'] + [mon.name for mon in self.mons])
        df.index.name = 'Ticker'
        
        while ticker < self.total_ticks:
            
            bars = [a + b for a, b in zip(bars, tick_step)]
            
            bar_max = max(bars)
            if bar_max >= self.bar_full:
                i = bars.index(bar_max)
                df.loc[str(ticker)] = [self.mons[i].name]+ [ int(bar2) for bar2 in bars]
                bars[i] = 0
            else:
                df.loc[str(ticker)] = [' ']+[ int(bar2) for bar2 in bars]
            ticker += 1
            
        return(df)

Set up your Arena monsters by providing base_speed, bonus_speed (from rune set and stats) and name for each of them.

In [74]:
kat = Monster(116,15,'kat')
jamire = Monster(113,4,'jamire')
chloe = Monster(111,143.75,'chloe')
chasun = Monster(101,43,'chasun')

Then set up your arena instance by providing speed_totem bonus and leader bonus.

In [79]:
spd_totem = 0.12
lead = 0.24
team1 = Team(spd_totem, lead, [chloe, chasun, kat, jamire],'AO')

Lastly simulate your Arena with your team and for 30 tickers!

In [82]:
arena = Arena(team1,30)
df = arena.simulate()
df

,Mover,chloe,chasun,kat,jamire
Ticker,,,,,
1,,20,12,12,11
2,,41,25,24,22
3,,61,37,36,33
4,,82,50,48,44
5,chloe,103,63,60,55
6,,20,75,72,66
7,,41,88,84,77
8,chasun,61,101,96,88
9,kat,82,12,108,99


As you can see Chloe takes the lead at 5th ticker (T5) and she pulls Fanatic (S3), which provides immunity and invincibility for the whole team. Then at T8, Chasun pulls S2 to provide attack buff, which is followed by Katarina who pulls Swords of Discharge, which makes three random shots, each of which ignores defends and delivers 30k damage when crits. Lastly Jamire takes the stage with Illusion of Time (S3) to reset the cooldowns of all mons, followed by Chloe right at the next ticker, which keeps the invincibility for four more tickers on everyone until she get the turn again. At T18, Katarina takes the stage again and delivers Swords of Discharge on more time.

Ideally with Chloe-(Buffer)-Kat-Jamire teams you want Chloe to be the fastest monster on the field to pull her Fanatic as early as she can. Chloe's S3 protects every ally so then you might leave some time for the opponent to move. Then Buffer buffs the attack in most comps and Katarina, your damage dealer does her trick with her S3. Then Jamire comes in to refresh cooldown with his S3 and Chloe comes next to pull her S3 to protect the team. Since Katarina is the most vulnerable mon in your comp, being the damage dealer, you want Jamire and Chloe take their turns as quickly as possible.

So the ideal turn order:

Chloe - leave some time - (Buffer) at T(X) - Kat at T(X+1) - Jamire at T(X+2) - Chloe at T(X+3)

As you can see above Team1 works greatly with X=8 when it is the only ream in Arena, which is never the case. :) Since your opponent's monsters can interfere with your monsters, their speeds matter too. 

Let's set up an opponent team to see how things would change with an actual opponent.

In [71]:
opp1 = Monster(110,15,'opp1')
opp2 = Monster(106,115,'opp2')
opp3 = Monster(103,15,'opp3')
opp4 = Monster(105,25,'opp4')

team2 = Team(0.15, 0.24 , [opp1, opp2, opp3, opp4],'AD')

arena = Arena(team1 ,team2,30)
df = arena.simulate()
df

,Mover,chloe,chasun,kat,jamire,o1,o2,o3,o4
1,,20,12,12,11,11,18,11,11
2,,41,25,24,22,23,36,22,23
3,,61,37,36,33,35,55,33,35
4,,82,50,48,44,47,73,44,47
5,chloe,103,63,60,55,58,91,55,59
6,o2,20,75,72,66,70,110,66,71
7,,41,88,84,77,82,18,77,83
8,chasun,61,101,96,88,94,36,88,95
9,kat,82,12,108,99,105,55,99,107
10,o4,103,25,12,110,117,73,110,119


With the introduction of the opponent team, we see that everything goes fine in the beginning. Chloe takes the lead and pulls Fanatic, which potentially saves the team (unless opp2 strips immunity and invincibility) and turn order is followed by Chasun and Katarina as desired. 

However from Ticker 10, things go not so much as expected. Since at T10, multiple monsters pass 100 bar, opponent monsters intercept Jamire. And since Jamire is very slow compared to Chloe, Chloe's bar overtakes Jamire's on T11 already. Therefore Chloe takes the turn before Jamire at T12 and when Jamire finally takes turn at T14, Chloe is only at 41% of its attack bar. So the team goes out of sync because of the interceptions from opponents 1,3 and 4. 

# Notes

This Arena Simulator / Speed Tuner does not factor in attack bar increase skills or speed buff/debuff effects, which I might add later. 

Also note that the tool can be used also for other modes such as TOA speed tuning with one team as the number of team you can add is not constrained. 

I consider making an html UI for this for less Python-savy users. Poke me for it please :)